# HW01
* 이름: 아무개
* 학번: 99990000

위에 이름과 학번을 자신의 것으로 기입하고 또 맨 위에 노트북의 제목도 `HW01<이름><학번>`으로 바꾸어 HiClass를 통해 마감시간까지 제출하면 됩니다.
마감시간은 HiClass에 공지되어 있습니다.

In [1]:
class RGB {
    int r, g, b;
    RGB() { r=0; g=0; b=0; }
    RGB(int _r, int _g, int _b) { r=_r; g=_g; b=_b; }
    @Override public String toString() { return "rgb("+r+","+g+","+b+")"; }
}

com.twosigma.beaker.javash.bkr31bc15ee.RGB

In [2]:
class Point {
    int x, y;
    Point() { x=0; y=0; }
    Point(int _x, int _y) { x=_x; y=_y; }
}

com.twosigma.beaker.javash.bkr31bc15ee.Point

In [3]:
// 연산용 
class Calc{

    static double Length(Point p1, Point p2) 
    {
        double x = p1.x - p2.x; 
        double y = p1.y - p2.y;
        return Math.sqrt(x*x+y*y);
    }
    static int Area(double lineA,double lineB,double lineC)
    {
        double length = (lineA + lineB + lineC) / 2;
        double area = Math.sqrt(length*(length-lineA)*(length-lineB)*(length-lineC));
        return (int)Math.round(area);
    }
    static boolean EqArea(Point a,Point b,Point c,Point p)
    {
        double lineab   = Length(a,b);
        double linebc   = Length(b,c);
        double lineac   = Length(a,c);
        double lineap   = Length(a,p);
        double linebp   = Length(b,p);
        double linecp   = Length(c,p);
        int areaabp = Area(lineap,linebp,lineab);
        int areabcp = Area(linecp,linebp,linebc);
        int areaacp = Area(lineap,linecp,lineac); 
        int areaabc = Area(lineab,linebc,lineac);
        if(areaabc == (areaabp+areabcp+areaacp)) // 내부에 점이 있다면 해당 점으로 만든 삼각형 3개의 넓이가 원래 삼각형과 같을것이므로
            return true;
        else
            return false;
    }
    static Point Shortlen(int x, int y,double px,double py, Point a,Point c,Point p)
    {
            //System.out.println(" "+px+","+py+" ");
            double min = 600; // 대충 화면보다 크게
            double minx=0,miny=0;
            double length = Math.sqrt(x*x+y*y);
            double xr = x/length;
            double yr = y/length;
            //System.out.println(""+xr+","+yr);
            double ix,iy,il;
            while((px >= Math.min(a.x,c.x) && px<= Math.max(a.x,c.x)) && (py >= Math.min(a.y,c.y) && py<=Math.max(a.y,c.y)) )
            {
                px += xr; //px,py : 현재 검사하고 있는 점. 초기값음 대각선 변의 끝점중 하나
                py += yr; 
                ix=px-p.x; // 검사할 점과 현재 점의 좌표차이 
                iy=py-p.y;
                il = Math.sqrt(ix*ix+iy*iy); // 길이
               // System.out.println("px "+px+", py"+py+" len "+ il);
                if(il<min){
                    min = il;
                    minx = px;
                    miny = py;
                }
            }
            Point mins = new Point((int)minx,(int)miny);
            return mins;
    }
}

com.twosigma.beaker.javash.bkr31bc15ee.Calc

## 첫번째 설계
* `CircObj`, `RectObj`, `RTriObj` 클래스 작성 (3점)
* 같은 클래스의 인스턴스끼리 (`CircObj`끼리, `RectObj`끼리, `RTriObj`) 충돌 검사 테스트 사례 작성 및 각 테스트 사례를 svg 출력으로 확인 (3점)
* 서로 다른 클래스의 (`CircObj`와 `RectObj`, `RectObj`와 `RTriObj`, `RTriObj`와 `CircObj`) 인스턴스 충돌 검사 테스트 사례 작성 및 각 테스트 사례를 svg 출력으로 확인 (3점)

In [4]:
abstract class ShapeObj { // 어떤 추상적인 모양의 물건 클래스
    Point p;   // 물건 위치에 대한 정보
    RGB color; // 물건의 색깔
    ShapeObj(Point p, RGB c) { this.p=p; color=c; }
    abstract Point closest(Point p); // 점 p와 가장 까까운 도형에 포함된 점 (소수점 이하는 적절히 처리해 정수 좌표로) //원 내부는 자기자신
    abstract boolean collides(Point p); // 점 p와 충돌하는가?
    abstract boolean collides(ShapeObj o); // o와 충돌하는가?
    abstract String toSVG(); // svg 태그 생성
    @Override
    public String toString() { return toSVG(); }
}

com.twosigma.beaker.javash.bkr31bc15ee.ShapeObj

In [5]:
class CircObj extends ShapeObj { // 원 모양 물건 클래스
    // 원의 경우 p는 중심이다
    int r; // 반지름
    CircObj(int r, Point p, RGB c) 
    {
        super(p, c); // 부모 클래스 생성자 호출
        this.r = r;
    }
    // 아래를 완성하라 (1점)
    @Override
    Point closest(Point p) // 점 p와 가장 까까운 도형에 포함된 점 
    { 
        double difX   = this.p.x - p.x; // x축 차
        double difY   = this.p.y - p.y; // y축 차
        double length = Math.sqrt(Math.pow(difX,2) + Math.pow(difY,2)); // 중심점과 점간의 거리
        if(r > length) // 반지름이 길이보다 큰 경우, 점이 원 내부에 존재하므로 해당 점을 리턴
            return p;
        else           // 반지름이 길이보다 작은 경우, 점이 외부에 위치하므로 원 경계의 좌표를 리턴
        {
            double angle   = Math.toRadians(Math.atan2(difY,difX) * (180.0/Math.PI)-90);
            // 라디안값을 구한 후 각도로 바꾸어 보정치 부여, 다시 라디안값으로 변환 (atan 사용시 사분면 전체 각도 불가)
            
            double borderx = this.p.x + r * Math.sin(angle); 
            double bordery = this.p.y - r * Math.cos(angle); // 디스플레이는 y축 방향이 반대이므로
            return new Point((int)Math.round(borderx),(int)Math.round(bordery)); // round 함수로 반올림하여 정수로
        }
    }
    @Override
    boolean collides(Point p) 
    { 
        Point p2 = closest(p);
        if(p2.x ==p.x && p2.y == p.y) //인자 p와 리턴값이 같다면 도형에 포함, 충돌
            return true;
        else
            return false;
    }
    @Override
    boolean collides(ShapeObj o) 
    { 
        Point po = closest(o.p); // 상대 중심과 내 도형의 위치(충돌-o.p,비충돌-내 도형의 경계) 저장 
        if(o.collides(po)==true) // 내 도형의 중심이나 상대 도형의 경계가 내 도형과 충돌한다면 츙돌
            return true;
        else
            return false;    
  
    }
    @Override
    String toSVG() 
    {
        return "<circle cx='"+p.x+"' cy='"+p.y+"' r='"+r+"' style='stroke:black; stroke-width:1; fill:"+color+"; opacity:0.33' />";
    }
}

com.twosigma.beaker.javash.bkr31bc15ee.CircObj

<!--import com.twosigma.beakerx.widget.*;

RGB red = new RGB(255,0,0);
RGB blue = new RGB(0,0,255);

CircObj c1 = new CircObj(30, new Point( 150, 150), red);
CircObj c2 = new CircObj(10, new Point(50,50), blue);
CircObj c3 = new CircObj(10, new Point(50,250), red);
CircObj c4 = new CircObj(10, new Point(250,250), blue);
CircObj c5 = new CircObj(10, new Point(250,50), red);

Point p1 = c1.closest(c2.p);
CircObj c6 = new CircObj(5, p1, new RGB(0,0,0));

Point p2 = c1.closest(c3.p);
CircObj c7 = new CircObj(5, p2, new RGB(255,0,0)); // R

Point p3 = c1.closest(c4.p);
CircObj c8 = new CircObj(5, p3, new RGB(0,255,0)); //G

Point p4 = c1.closest(c5.p);
CircObj c9 = new CircObj(5, p4, new RGB(0,0,255)); //B



HTML widget = new HTML();
String sgv =
    "<svg width='300' height='300' style='border:solid 1px'>"
    + c1 + c2 + c3 + c4 + c5 +c6 + c7 + c8 + c9 +
    "</svg>";
widget.setValue("<div>"+sgv+"</div>");
return widget; // 그림에 나타나는 것과 맞는지 시각적으로 확인해 볼 수 있도록 toSVG를 정의한 것이다
-->

<!--
import com.twosigma.beakerx.widget.*;
int x1 = 30,x2 = 20;
int y1 = 40,y2 = 50;
CircObj c1 = new CircObj(20, new Point(x1,y1), new RGB(255,0,0));
CircObj c2 = new CircObj(1, new Point(x2,y2), new RGB(0,255,0));
/*
double x = x1 - x2;
double y = y1 - y2;
double l = Math.sqrt(x*x+y*y);

double angle = Math.atan(y/x) * (180.0/Math.PI);
System.out.println(l);
System.out.println(angle);

double borderx = x1 + 20 * Math.cos(Math.toRadians(angle));
double bordery = y1 - 20 * Math.sin(Math.toRadians(angle));
CircObj c3 = new CircObj(1, new Point((int)Math.round(borderx),(int)Math.round(bordery)), new RGB(0,0,0));
System.out.println(borderx + "," + bordery);
*/

HTML widget = new HTML();
String sgv =
    "<svg width='100' height='100' style='border:solid 1px'>"
    + c1 +c2 +
    "</svg>";
widget.setValue("<div>"+sgv+"</div>");
System.out.println(c1.collides(c2));

//System.out.println(c1 instanceof CircObj);
//System.out.println(c1 instanceof ShapeObj);
return widget; // 그림에 나타나는 것과 맞는지 시각적으로 확인해 볼 수 있도록 toSVG를 정의한 것이다


-->

<!--
import com.twosigma.beakerx.widget.*;
int x1 = 30,x2 = 40;
int y1 = 40,y2 = 60;
CircObj c1 = new CircObj(20, new Point(x1,y1), new RGB(255,0,0));
CircObj c2 = new CircObj(10, new Point(x2,y2), new RGB(0,255,0));
CircObj c3 = new CircObj(2, c1.closest(c2.p), new RGB(0,0,0));

HTML widget = new HTML();
String sgv =
    "<svg width='100' height='100' style='border:solid 1px'>"
    + c1 +c2 + c3+
    "</svg>";
widget.setValue("<div>"+sgv+"</div>");

System.out.println(c1.collides(c2));

return widget; 
-->

In [6]:
import com.twosigma.beakerx.widget.*;

RGB red = new RGB(255,0,0);
RGB blue = new RGB(0,0,255);

CircObj c1 = new CircObj(40, new Point( 50, 50), red);
CircObj c2 = new CircObj(70, new Point(110,110), blue);
CircObj c3 = new CircObj(40, new Point(160,160), red);
CircObj c4 = new CircObj(70, new Point(220,220), blue);

System.out.println( c1.toSVG() );
System.out.println( c2.toSVG() );
System.out.println( c3.toSVG() );
System.out.println( c4.toSVG() );

System.out.println( c1.collides(c2) ); // 이런 결과가
System.out.println( c2.collides(c3) ); // 이런 결과가
System.out.println( c3.collides(c4) ); // 이런 결과가

System.out.println( c1.collides(c3) ); // 요런 결과가
System.out.println( c2.collides(c4) ); // 요런 결과가

HTML widget = new HTML();
String sgv =
    "<svg width='300' height='300' style='border:solid 1px'>"
    + c1 + c2 + c3 + c4 +
    "</svg>";
widget.setValue("<div>"+sgv+"</div>");
return widget; // 그림에 나타나는 것과 맞는지 시각적으로 확인해 볼 수 있도록 toSVG를 정의한 것이다

<circle cx='50' cy='50' r='40' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' />
<circle cx='110' cy='110' r='70' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' />
<circle cx='160' cy='160' r='40' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' />
<circle cx='220' cy='220' r='70' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' />
true
true
true
false
false


In [7]:
class RectObj extends ShapeObj { // 직사각형 모양 물건 클래스
    // 직사각형의 경우 경우 p는 왼쪽 위 꼭지점이다
    int w; // 너비 (양수)
    int h; // 높이 (양수)

    RectObj(int w, int h, Point p, RGB c) 
    {
        super(p, c); // 부모 클래스 생성자 호출
        this.w = w;
        this.h = h;
    }
    @Override
    Point closest(Point p) 
    { // 점 p와 가장 까까운 도형에 포함된 점 (소수점 이하는 적절히 처리해 정수 좌표로)
        //case 8
        if(p.x < this.p.x && p.y < this.p.y) // 왼쪽 위 모서리
            return new Point(this.p.x,this.p.y);
        else if(p.x < this.p.x && p.y > this.p.y + this.h) // 왼쪽 아래 모서리
            return new Point(this.p.x,this.p.y+this.h);
        else if(p.x > this.p.x + this.w && p.y > this.p.y + this.h) // 오른쪽 아래 모서리
            return new Point(this.p.x+this.w,this.p.y+this.h);
        else if(p.x > this.p.x + this.w && p.y < this.p.y ) // 오른쪽 위 모서리
            return new Point(this.p.x + this.w, p.y);
        else if((p.x < this.p.x || p.x > this.p.x + this.w) && p.y >= this.p.y && p.y <= this.p.y + this.h) // 좌우 바깥
            return new Point(this.p.x,p.y);
        else if(p.x >= this.p.x && p.x <= this.p.x + this.w && (p.y < this.p.y||p.y > this.p.y + this.h)) // 상하 바깥
            return new Point(p.x,this.p.y);
        else // 모두 아니면 좌표 안에 있으므로 해당 점 리턴 
            return p;
    }
    @Override
    boolean collides(Point p) 
    { 
        Point p2 = closest(p);
        if(p2.x ==p.x && p2.y == p.y) //인자 p와 리턴값이 같다면 도형에 포함, 충돌
            return true;
        else
            return false;
    }
    @Override
    boolean collides(ShapeObj o) 
    { 
        Point po = closest(o.p); // 상대 중심과 내 도형의 위치(충돌-o.p,비충돌-내 도형의 경계) 저장 
        if(o.collides(po)==true) // 내 도형의 중심이나 상대 도형의 경계가 내 도형과 충돌한다면 츙돌
            return true;
        else
            return false; 
    }
    @Override
    String toSVG() 
    {
        return "<rect x='"+p.x+"' y='"+p.y+"' width='"+w+"'height='"+h+"' style='stroke:black; stroke-width:1; fill:"+color+"; opacity:0.33' />";
    }
}

com.twosigma.beaker.javash.bkr31bc15ee.RectObj

In [8]:
import com.twosigma.beakerx.widget.*;

int x1 = 40, x2 = 60, x3 = 100, x4 = 110;
int y1 = 40, y2 = 50, y3 = 90,  y4 = 130;


RectObj r1 = new RectObj(30, 40, new Point( x1,y1),new RGB(0,255,255));
RectObj r2 = new RectObj(40, 40, new Point( x2,y2),new RGB(255,0,255));
RectObj r3 = new RectObj(30, 100, new Point( x3,y3),new RGB(255,0,0));
RectObj r4 = new RectObj(100, 30, new Point( x4,y4),new RGB(0,0,255));
CircObj c1 = new CircObj(30, new Point(x4,x4) ,new RGB(0,255,0));
System.out.println( r1.collides(r2) ); // 이런 결과가
System.out.println( r2.collides(r3) ); // 이런 결과가
System.out.println( r3.collides(r4) ); // 이런 결과가

System.out.println( r1.collides(r3) ); // 이런 결과가
System.out.println( r2.collides(r4) ); // 이런 결과가
System.out.println(  "c.충돌");
System.out.println( c1.collides(r1) );
System.out.println( c1.collides(r2) );
System.out.println( c1.collides(r3) );
System.out.println( c1.collides(r4) );
System.out.println(  "r.충돌");
System.out.println( r1.collides(c1) );
System.out.println( r2.collides(c1) );
System.out.println( r3.collides(c1) );
System.out.println( r4.collides(c1) );




HTML widget = new HTML();
String sgv =
    "<svg width='300' height='300' style='border:solid 1px'>"
    + r1 + r2 + r3 + r4 + c1 + 
    "</svg>";
widget.setValue("<div>"+sgv+"</div>");


return widget; // 그림에 나타나는 것과 맞는지 시각적으로 확인해 볼 수 있도록 toSVG를 정의한 것이다

true
true
true
false
false
c.충돌
false
true
true
true
r.충돌
false
true
true
true


In [9]:
class RTriObj extends ShapeObj { // 직각삼각형 모양 물건 클래스
    // Point p; // 직각인 꼭지점
    int w; // 직각인 꼭지점으로부터 음수/양수로 왼쪽/오른쪽 방향으로의 w만큼 떨어진 꼭지점 나타냄
    int h; // 직각인 꼭지점으로부터 음수/양수로 아래쪽/위쪽 방향으로의 h만큼 떨어진 꼭지점 나타냄
    
    // CircObj처럼 나머지를 완성하라 (1점)
    RTriObj(int w, int h, Point p, RGB c)  
    {
        super(p, c); // 부모 클래스 생성자 호출
        this.w = w;
        this.h = h;
    }
    
    @Override
    Point closest(Point p) 
    { // 점 p와 가장 까까운 도형에 포함된 점 (소수점 이하는 적절히 처리해 정수 좌표로)
        Point a = new Point (this.p.x+w,this.p.y); // x축 변화
        Point c = new Point (this.p.x,this.p.y+h); // y축 변화
        int maxX = Math.max(this.p.x, this.p.x+w),minX = Math.min(this.p.x, this.p.x+w); 
        int maxY = Math.max(this.p.y, this.p.y+h),minY = Math.min(this.p.y, this.p.y+h);
        int x,y; // 대각선 변 연산시 사용
        double py,px; //대각선 변 연산시 사용2(초기 반복문 시작위치)
        if(this.p.x>this.p.x+w && this.p.y>this.p.y+h) // 좌/상 방향으로 뻗은 삼각형(-,-) / 대각선모양
        {   //        c(+h)
            // a(+w)  b(this.p)
            if(p.x > maxX && (p.y < maxY && p.y > minY) ) // 오른쪽 바깥
                return new Point(maxX, p.y);
            else if( (p.x > minX && p.x < maxX) && p.y > maxY) // 아래쪽 바깥
                return new Point(p.x, maxY);
            else if( p.x >= maxX && p.y >= maxY) // 오른쪽 아래 모서리
                return this.p;
            else if( p.x >= maxX && p.y <= minY) // 오른쪽 위 모서리
                return c;
            else if( p.x <= minX && p.y >= maxY) // 왼쪽 아래 모서리
                return a;
            else if(Calc.EqArea(a,this.p,c,p)==true) // 내부
                return p;
            else 
            {   //내부가 아니면 결국 대각선이므로 대각선 검사용 세팅
                // / 방향은 아래에서 시작시 x+,y-
                x = c.x - a.x;
                y = c.y - a.y;
                px = a.x;
                py = a.y;
                return Calc.Shortlen(x,y,px,py,a,c,p);
            }
        }
        else if(this.p.x>this.p.x+w && this.p.y<this.p.y+h) // 좌/하 방향으로 뻗은 삼각형(-,+)
        {   // a(+w)  b(this.p)
            //        c(+h)
            if(p.x > maxX && (p.y > minY && p.y < maxY) ) // 오른쪽 바깥
                return new Point(maxX, p.y);
            else if( (p.x > minX && p.x < maxX) && p.y < minY) // 위쪽 바깥
                return new Point(p.x, minY);
            else if( p.x >= maxX && p.y >= maxY) // 오른쪽 아래 모서리
                return c;
            else if( p.x >= maxX && p.y <= minY) // 오른쪽 위 모서리
                return this.p;
            else if( p.x <= minX && p.y <= minY) // 왼쪽 위 모서리
                return a;
            else if(Calc.EqArea(a,this.p,c,p)==true) // 내부
                return p;
            else 
            { 
                //내부가 아니면 결국 대각선이므로 대각선 검사용 세팅
                // \ 방향은 왼쪽에서 시작시 x+,y+
                x = c.x - a.x;
                y = c.y - a.y;
                px = a.x;
                py = a.y;
                return Calc.Shortlen(x,y,px,py,a,c,p);
            }
        }
        else if(this.p.x<this.p.x+w && this.p.y<this.p.y+h) // 하/우 방향으로 뻗은 삼각형(+,+)
        {
            //   b(this.p) a(+w)
            //   c(+h)
            if(p.x < minX && (p.y > minY && p.y < maxY) ) // 왼쪽 바깥
                return new Point(minX, p.y);
            else if( (p.x > minX && p.x < maxX) && p.y < minY) // 위쪽 바깥
                return new Point(p.x, minY);
            else if( p.x <= minX && p.y >= maxY) // 왼쪽 아래 모서리
                return c;
            else if( p.x >= maxX && p.y <= minY) // 오른쪽 위 모서리
                return a;
            else if( p.x <= minX && p.y <= minY) // 왼쪽 위 모서리
                return this.p;
            else if(Calc.EqArea(a,this.p,c,p)==true) // 내부
                return p;
            else 
            { 
                //내부가 아니면 결국 대각선이므로 대각선 검사용 세팅
                // / 방향은 왼쪽에서 시작시 x+,y-
                x = a.x - c.x;
                y = a.y - c.y;
                px = c.x;
                py = c.y;
                return Calc.Shortlen(x,y,px,py,a,c,p);
            }
        }
        else  // 상/우 방향으로 뻗은 삼각형(+,-)
        {   //   c(+h)
            //   b(this.p) a(+w)
            if(p.x < minX && (p.y > minY && p.y < maxY) ) // 왼쪽 바깥
                return new Point(minX, p.y);
            else if( (p.x > minX && p.x < maxX) && p.y > maxY) // 아래쪽 바깥
                return new Point(p.x, maxY);
            else if( p.x <= minX && p.y >= maxY) // 왼쪽 아래 모서리
                return this.p;
            else if( p.x >= maxX && p.y <= maxY) // 오른쪽 아래 모서리
                return a;
            else if( p.x <= minX && p.y <= minY) // 왼쪽 위 모서리
                return c;
            else if(Calc.EqArea(a,this.p,c,p)==true) // 내부
                return p;
            else 
            { 
                //내부가 아니면 결국 대각선이므로 대각선 검사용 세팅
                // \ 방향은 왼쪽에서 시작시 x+,y+
                x = a.x - c.x;
                y = a.y - c.y;
                px = c.x;
                py = c.y;
                return Calc.Shortlen(x,y,px,py,a,c,p);
            }
        }    
        
    }
    @Override
    boolean collides(Point p) 
    { 
        Point p2 = closest(p);
        if(p2.x ==p.x && p2.y == p.y) //인자 p와 리턴값이 같다면 도형에 포함, 충돌
            return true;
        else
            return false;
    }
    @Override
    boolean collides(ShapeObj o) 
    { 
        Point po1 = closest(o.p); // 상대 중심과 내 도형의 위치(충돌-o.p,비충돌-내 도형의 경계) 저장 
        Point po2 = o.closest(p);
        if(o.collides(po1) || collides(po2)) // 내 도형의 중심이나 상대 도형의 경계가 내 도형과 충돌한다면 츙돌
            return true;
        else
            return false; 
    }
    @Override
    String toSVG() 
    { 
        return "<polygon points='"+p.x+","+p.y+" "+ (p.x+w)+","+p.y +" "+p.x+","+(p.y +h)+"' style='stroke:black; stroke-width:1; fill:"+color+"; opacity:0.33' />";
    }
}

com.twosigma.beaker.javash.bkr31bc15ee.RTriObj

<!--import com.twosigma.beakerx.widget.*;
RTriObj t1 = new RTriObj(-30, 70, new Point(100,100),new RGB(0,255,255)); // 70, 100 & 100 , 170

Point p1 =  new Point(50,50);
Point p2 =  new Point(50,250);
Point p3 =  new Point(250,250);
Point p4 =  new Point(250,50);
Point p5 =  new Point(90,120);
CircObj c1 = new CircObj(5, t1.closest(p1), new RGB(0,0,255));
CircObj c2 = new CircObj(5, t1.closest(p2), new RGB(0,255,255)); //
CircObj c3 = new CircObj(5, t1.closest(p3), new RGB(255,0,255));
CircObj c4 = new CircObj(5, t1.closest(p4), new RGB(255,0,0));
CircObj c9 = new CircObj(5, t1.closest(p5), new RGB(255,255,0));
CircObj c5 = new CircObj(5, p1, new RGB(0,0,255));
CircObj c6 = new CircObj(5, p2, new RGB(0,255,255)); //
CircObj c7 = new CircObj(5, p3, new RGB(255,0,255));
CircObj c8 = new CircObj(5, p4, new RGB(255,0,0));
CircObj c10 = new CircObj(5, p5, new RGB(255,255,0));
System.out.println(t1.collides(t1.closest(p1)));
System.out.println(t1.collides(t1.closest(p2)));
System.out.println(t1.collides(t1.closest(p3)));
System.out.println(t1.collides(t1.closest(p4)));
System.out.println(t1.collides(t1.closest(p5)));
HTML widget = new HTML();
String sgv =
    "<svg width='300' height='300' style='border:solid 1px'>"
    + t1 + c1  + c2 + c3 + c4 + c5 + c6  + c7 +c8+c9+c10+
    "</svg>";
widget.setValue("<div>"+sgv+"</div>");


return widget; // 그림에 나타나는 것과 맞는지 시각적으로 확인해 볼 수 있도록 toSVG를 정의한 것이다 */ -->

In [10]:
import com.twosigma.beakerx.widget.*;

int x1 = 50, x2 = 100, x3 = 150, x4 = 110;
int y1 = 50, y2 = 100, y3 = 150, y4 = 130;

System.out.println("1:하늘 2:빨강 3:초록 4:파랑 5:보라");
RTriObj r1 = new RTriObj(50, 40, new Point( x1,y1),new RGB(0,255,255));
RTriObj r2 = new RTriObj(40, -60, new Point( x2,y2),new RGB(255,0,0));
RTriObj r3 = new RTriObj(-70, -40, new Point( x3,y3),new RGB(0,255,0));
RTriObj r4 = new RTriObj(-100, 30, new Point( x4,y4),new RGB(0,0,255));
CircObj c1 = new CircObj(30, new Point(x4,x4) ,new RGB(0,255,0));
RectObj r5 = new RectObj(30,30, new Point(x2,x2), new RGB(200,200,200));

RTriObj r6 = new RTriObj(60,60, new Point(70,70),new RGB(255,0,255));
System.out.println("하늘/빨강"+ r1.collides(r2) ); // 이런 결과가
System.out.println("빨강/초록"+ r2.collides(r3) ); // 이런 결과가
System.out.println("초록/파랑"+ r3.collides(r4) ); // 이런 결과가
System.out.println("파랑/보라"+ r4.collides(r6) ); // 이런 결과가

System.out.println("하늘/초록"+ r1.collides(r3) ); // 이런 결과가
System.out.println("빨강/파랑"+ r2.collides(r4) ); // 이런 결과가
System.out.println("초록/보라"+ r3.collides(r6) ); // 이런 결과가
System.out.println(  "원->삼각형.충돌");
System.out.println( c1.collides(r1) );
System.out.println( c1.collides(r2) );
System.out.println( c1.collides(r3) );
System.out.println( c1.collides(r4) );
System.out.println( c1.collides(r6) );
System.out.println(  "삼각형->원.충돌");
System.out.println( r1.collides(c1) );
System.out.println( r2.collides(c1) );
System.out.println( r3.collides(c1) );
System.out.println( r4.collides(c1) );
System.out.println( r6.collides(c1) );




System.out.println(  "삼각형->사각형.충돌");
System.out.println( r1.collides(r5) );
System.out.println( r2.collides(r5) );
System.out.println( r3.collides(r5) );
System.out.println( r4.collides(r5) );
System.out.println( r6.collides(r5) );

System.out.println(  "사각형->삼각형.충돌");
System.out.println( r5.collides(r1) );
System.out.println( r5.collides(r2) );
System.out.println( r5.collides(r3) );
System.out.println( r5.collides(r4) );
System.out.println( r5.collides(r6) );


HTML widget = new HTML();
String sgv =
    "<svg width='300' height='300' style='border:solid 1px'>"
    + r1 + r2 + r3 + r4 + c1 + r5 + r6 +
    "</svg>";
widget.setValue("<div>"+sgv+"</div>");


return widget; // 그림에 나타나는 것과 맞는지 시각적으로 확인해 볼 수 있도록 toSVG를 정의한 것이다

1:하늘 2:빨강 3:초록 4:파랑 5:보라
하늘/빨강true
빨강/초록false
초록/파랑true
파랑/보라true
하늘/초록false
빨강/파랑false
초록/보라false
원->삼각형.충돌
false
true
true
true
true
삼각형->원.충돌
false
true
true
true
true
삼각형->사각형.충돌
false
true
true
true
true
사각형->삼각형.충돌
false
true
true
true
true


## 두번째 설계
첫번째 설계와 같은 문제를 해결하지만 조금 다른 방식의 설계로 프로그램을 재작성해 보라.
* `Circle`, `Rectangle`, `RTriangle` 클래스를 작성하라 (1점)
* 첫번째에서 했던 같은 테스트 사례들로 모두 실행해 보라 (3점)

In [11]:
abstract class Shape { // 어떤 추상적인 모양 클래스
    RGB color; // 물건의 색깔
    Shape(RGB c) { color=c; }
    abstract String toSVG(Point self);
    abstract Point closest(Point self, Point p);
    abstract boolean collides(Point self, Point p);
    abstract boolean collides(Point self, Obj o);
}

class Obj {
    Point p;
    Shape s; // Shape이 Obj의 속성이다
    Obj(Point _p, Shape _s) { p=_p; s=_s; }
    Point closest(Point p) { return s.closest(this.p, p); } // 점 p와 가장 까까운 도형에 포함된 점 (소수점 이하는 적절히 처리해 정수 좌표로)
    boolean collides(Point p) { return s.collides(this.p, p); } // 점 p와 충돌하는가?
    boolean collides(Obj o) { return s.collides(this.p, o); } // o와 충돌하는가
    @Override
    public String toString() { return s.toSVG(this.p); }
}

com.twosigma.beaker.javash.bkr31bc15ee.Shape

In [12]:
class Circle extends Shape { // 원 모양 클래스
    int r;
    Circle(int r, RGB c) { super(c); this.r = r; }
    
    @Override
    Point closest(Point self, Point p) {
        double difX   = self.x - p.x; // x축 차
        double difY   = self.y - p.y; // y축 차
        double length = Math.sqrt(Math.pow(difX,2) + Math.pow(difY,2)); // 중심점과 점간의 거리
        if(r > length) // 반지름이 길이보다 큰 경우, 점이 원 내부에 존재하므로 해당 점을 리턴
            return p;
        else           // 반지름이 길이보다 작은 경우, 점이 외부에 위치하므로 원 경계의 좌표를 리턴
        {
            double angle   = Math.toRadians(Math.atan2(difY,difX) * (180.0/Math.PI)-90);
            // 라디안값을 구한 후 각도로 바꾸어 보정치 부여, 다시 라디안값으로 변환 (atan 사용시 사분면 전체 각도 불가)
            
            double borderx = self.x + r * Math.sin(angle); 
            double bordery = self.y - r * Math.cos(angle); // 디스플레이는 y축 방향이 반대이므로
            return new Point((int)Math.round(borderx),(int)Math.round(bordery)); // round 함수로 반올림하여 정수로
        }    
    }
    @Override
    boolean collides(Point self, Point p) {
        Point p2 = closest(self,p);
        if(p2.x ==p.x && p2.y == p.y) //인자 p와 리턴값이 같다면 도형에 포함, 충돌
            return true;
        else
            return false;
    }
    @Override
    boolean collides(Point self, Obj o) {
        Point po = closest(self,o.p); // 상대 중심과 내 도형의 위치(충돌-o.p,비충돌-내 도형의 경계) 저장 
        if(o.collides(po)==true) // 내 도형의 중심이나 상대 도형의 경계가 내 도형과 충돌한다면 츙돌
            return true;
        else
            return false;    
    }
    @Override
    String toSVG(Point self) {
        return "<circle cx='"+self.x+"' cy='"+self.y+"' r='"+r+"' style='stroke:black; stroke-width:1; fill:"+color+"; opacity:0.33' />";
    }
}

com.twosigma.beaker.javash.bkr31bc15ee.Circle

In [13]:
import com.twosigma.beakerx.widget.*;

RGB red = new RGB(255,0,0);
RGB blue = new RGB(0,0,255);
RGB magenta = new RGB(255,0,255);

Circle s1 = new Circle(40, red);
Circle s2 = new Circle(70, blue);

Obj c1 = new Obj(new Point( 50, 50), s1);
Obj c2 = new Obj(new Point(110,110), s2);
Obj c3 = new Obj(new Point(160,160), s1);
Obj c4 = new Obj(new Point(220,220), s2);

System.out.println( c1 );
System.out.println( c2 );
System.out.println( c3 );
System.out.println( c4 );

System.out.println( c1.collides(c2) ); // 이런 결과가
System.out.println( c2.collides(c3) ); // 이런 결과가
System.out.println( c3.collides(c4) ); // 이런 결과가

System.out.println( c1.collides(c3) ); // 요런 결과가
System.out.println( c2.collides(c4) ); // 요런 결과가

HTML widget = new HTML();
String sgv1 =
    "<svg width='300' height='300' style='border:solid 1px'>"
    + c1 + c2 + c3 + c4 +
    "</svg>";

// s1 = new Circle(60, magenta);
s1.color = magenta; s1.r = 50;

String sgv2 = 
    "<svg width='300' height='300' style='border:solid 1px'>"
    + c1 + c2 + c3 + c4 +
    "</svg>";
widget.setValue("<div>"+sgv1+"</div>"+"<div>"+sgv2+"</div>");
return widget; // 그림에 나타나는 것과 맞는지 시각적으로 확인해 볼 수 있도록 toSVG를 정의한 것이다

<circle cx='50' cy='50' r='40' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' />
<circle cx='110' cy='110' r='70' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' />
<circle cx='160' cy='160' r='40' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' />
<circle cx='220' cy='220' r='70' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' />
true
true
true
false
false


In [14]:
class Rectangle extends Shape { // 직서각형 모양 클래스
    int w; // 너비 (양수)
    int h; // 높이 (양수)
    Rectangle(int w,int h, RGB c) {super(c); this.w = w; this.h = h;}
    
    @Override
    Point closest(Point self, Point p) 
    { // 점 p와 가장 까까운 도형에 포함된 점 (소수점 이하는 적절히 처리해 정수 좌표로)
        //case 8
        if(p.x < self.x && p.y < self.y) // 왼쪽 위 모서리
            return new Point(self.x,self.y);
        else if(p.x < self.x && p.y > self.y + this.h) // 왼쪽 아래 모서리
            return new Point(self.x,self.y+this.h);
        else if(p.x > self.x + this.w && p.y > self.y + this.h) // 오른쪽 아래 모서리
            return new Point(self.x+this.w,self.y+this.h);
        else if(p.x > self.x + this.w && p.y < self.y ) // 오른쪽 위 모서리
            return new Point(self.x + this.w, p.y);
        else if((p.x < self.x || p.x > self.x + this.w) && p.y >= self.y && p.y <= self.y + this.h) // 좌우 바깥
            return new Point(self.x,p.y);
        else if(p.x >= self.x && p.x <= self.x + this.w && (p.y < self.y||p.y > self.y + this.h)) // 상하 바깥
            return new Point(p.x,self.y);
        else // 모두 아니면 좌표 안에 있으므로 해당 점 리턴 
            return p;
    }
    @Override
    boolean collides(Point self, Point p) {
        Point p2 = closest(self,p);
        if(p2.x ==p.x && p2.y == p.y) //인자 p와 리턴값이 같다면 도형에 포함, 충돌
            return true;
        else
            return false;
    }
    @Override
    boolean collides(Point self, Obj o) {
        Point po = closest(self,o.p); // 상대 중심과 내 도형의 위치(충돌-o.p,비충돌-내 도형의 경계) 저장 
        if(o.collides(po)==true) // 내 도형의 중심이나 상대 도형의 경계가 내 도형과 충돌한다면 츙돌
            return true;
        else
            return false;    
    }
    @Override
    String toSVG(Point self) 
    {
        return "<rect x='"+self.x+"' y='"+self.y+"' width='"+w+"'height='"+h+"' style='stroke:black; stroke-width:1; fill:"+color+"; opacity:0.33' />";
    }
}

com.twosigma.beaker.javash.bkr31bc15ee.Rectangle

In [15]:
import com.twosigma.beakerx.widget.*;

RGB red = new RGB(255,0,0);
RGB blue = new RGB(0,0,255);
RGB magenta = new RGB(255,0,255);

Rectangle s1 = new Rectangle(60,60, red);
Rectangle s2 = new Rectangle(70,70, blue);

Obj c1 = new Obj(new Point( 50, 50), s1);
Obj c2 = new Obj(new Point(110,110), s2);
Obj c3 = new Obj(new Point(160,160), s1);
Obj c4 = new Obj(new Point(220,220), s2);

System.out.println( c1 );
System.out.println( c2 );
System.out.println( c3 );
System.out.println( c4 );

System.out.println( c1.collides(c2) ); // 이런 결과가
System.out.println( c2.collides(c3) ); // 이런 결과가
System.out.println( c3.collides(c4) ); // 이런 결과가

System.out.println( c1.collides(c3) ); // 요런 결과가
System.out.println( c2.collides(c4) ); // 요런 결과가

HTML widget = new HTML();
String sgv1 =
    "<svg width='300' height='300' style='border:solid 1px'>"
    + c1 + c2 + c3 + c4 +
    "</svg>";

// s1 = new Circle(60, magenta);

widget.setValue("<div>"+sgv1+"</div>");
return widget; // 그림에 나타나는 것과 맞는지 시각적으로 확인해 볼 수 있도록 toSVG를 정의한 것이다

<rect x='50' y='50' width='60'height='60' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' />
<rect x='110' y='110' width='70'height='70' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' />
<rect x='160' y='160' width='60'height='60' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' />
<rect x='220' y='220' width='70'height='70' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' />
true
true
true
false
false


In [16]:
class RTriangle extends Shape {
    int w; // 직각인 꼭지점으로부터 음수/양수로 왼쪽/오른쪽 방향으로의 w만큼 떨어진 꼭지점 나타냄
    int h; // 직각인 꼭지점으로부터 음수/양수로 아래쪽/위쪽 방향으로의 h만큼 떨어진 꼭지점 나타냄
    
    RTriangle(int w, int h, RGB c)  
    {
        super(c); // 부모 클래스 생성자 호출
        this.w = w;
        this.h = h;
    }
    @Override
    Point closest(Point self, Point p) 
    { // 점 p와 가장 까까운 도형에 포함된 점 (소수점 이하는 적절히 처리해 정수 좌표로)
        Point a = new Point (self.x+w,self.y); // x축 변화
        Point c = new Point (self.x,self.y+h); // y축 변화
        int maxX = Math.max(self.x, self.x+w),minX = Math.min(self.x, self.x+w); 
        int maxY = Math.max(self.y, self.y+h),minY = Math.min(self.y, self.y+h);
        int x,y; // 대각선 변 연산시 사용
        double py,px; //대각선 변 연산시 사용2(초기 반복문 시작위치)
        if(self.x>self.x+w && self.y>self.y+h) // 좌/상 방향으로 뻗은 삼각형(-,-) / 대각선모양
        {   //        c(+h)
            // a(+w)  b(self)
            if(p.x > maxX && (p.y < maxY && p.y > minY) ) // 오른쪽 바깥
                return new Point(maxX, p.y);
            else if( (p.x > minX && p.x < maxX) && p.y > maxY) // 아래쪽 바깥
                return new Point(p.x, maxY);
            else if( p.x >= maxX && p.y >= maxY) // 오른쪽 아래 모서리
                return self;
            else if( p.x >= maxX && p.y <= minY) // 오른쪽 위 모서리
                return c;
            else if( p.x <= minX && p.y >= maxY) // 왼쪽 아래 모서리
                return a;
            else if(Calc.EqArea(a,self,c,p)==true) // 내부
                return p;
            else 
            {   //내부가 아니면 결국 대각선이므로 대각선 검사용 세팅
                // / 방향은 아래에서 시작시 x+,y-
                x = c.x - a.x;
                y = c.y - a.y;
                px = a.x;
                py = a.y;
                return Calc.Shortlen(x,y,px,py,a,c,p);
            }
        }
        else if(self.x>self.x+w && self.y<self.y+h) // 좌/하 방향으로 뻗은 삼각형(-,+)
        {   // a(+w)  b(self)
            //        c(+h)
            if(p.x > maxX && (p.y > minY && p.y < maxY) ) // 오른쪽 바깥
                return new Point(maxX, p.y);
            else if( (p.x > minX && p.x < maxX) && p.y < minY) // 위쪽 바깥
                return new Point(p.x, minY);
            else if( p.x >= maxX && p.y >= maxY) // 오른쪽 아래 모서리
                return c;
            else if( p.x >= maxX && p.y <= minY) // 오른쪽 위 모서리
                return self;
            else if( p.x <= minX && p.y <= minY) // 왼쪽 위 모서리
                return a;
            else if(Calc.EqArea(a,self,c,p)==true) // 내부
                return p;
            else 
            { 
                //내부가 아니면 결국 대각선이므로 대각선 검사용 세팅
                // \ 방향은 왼쪽에서 시작시 x+,y+
                x = c.x - a.x;
                y = c.y - a.y;
                px = a.x;
                py = a.y;
                return Calc.Shortlen(x,y,px,py,a,c,p);
            }
        }
        else if(self.x<self.x+w && self.y<self.y+h) // 하/우 방향으로 뻗은 삼각형(+,+)
        {
            //   b(self) a(+w)
            //   c(+h)
            if(p.x < minX && (p.y > minY && p.y < maxY) ) // 왼쪽 바깥
                return new Point(minX, p.y);
            else if( (p.x > minX && p.x < maxX) && p.y < minY) // 위쪽 바깥
                return new Point(p.x, minY);
            else if( p.x <= minX && p.y >= maxY) // 왼쪽 아래 모서리
                return c;
            else if( p.x >= maxX && p.y <= minY) // 오른쪽 위 모서리
                return a;
            else if( p.x <= minX && p.y <= minY) // 왼쪽 위 모서리
                return self;
            else if(Calc.EqArea(a,self,c,p)==true) // 내부
                return p;
            else 
            { 
                //내부가 아니면 결국 대각선이므로 대각선 검사용 세팅
                // / 방향은 왼쪽에서 시작시 x+,y-
                x = a.x - c.x;
                y = a.y - c.y;
                px = c.x;
                py = c.y;
                return Calc.Shortlen(x,y,px,py,a,c,p);
            }
        }
        else  // 상/우 방향으로 뻗은 삼각형(+,-)
        {   //   c(+h)
            //   b(self) a(+w)
            if(p.x < minX && (p.y > minY && p.y < maxY) ) // 왼쪽 바깥
                return new Point(minX, p.y);
            else if( (p.x > minX && p.x < maxX) && p.y > maxY) // 아래쪽 바깥
                return new Point(p.x, maxY);
            else if( p.x <= minX && p.y >= maxY) // 왼쪽 아래 모서리
                return self;
            else if( p.x >= maxX && p.y <= maxY) // 오른쪽 아래 모서리
                return a;
            else if( p.x <= minX && p.y <= minY) // 왼쪽 위 모서리
                return c;
            else if(Calc.EqArea(a,self,c,p)==true) // 내부
                return p;
            else 
            { 
                //내부가 아니면 결국 대각선이므로 대각선 검사용 세팅
                // \ 방향은 왼쪽에서 시작시 x+,y+
                x = a.x - c.x;
                y = a.y - c.y;
                px = c.x;
                py = c.y;
                return Calc.Shortlen(x,y,px,py,a,c,p);
            }
        }
    }
    @Override
    boolean collides(Point self, Point p) 
    { 
        Point p2 = closest(self,p);
        if(p2.x ==p.x && p2.y == p.y) //인자 p와 리턴값이 같다면 도형에 포함, 충돌
            return true;
        else
            return false;
    }
    @Override
    boolean collides(Point self, Obj o) 
    { 
        Point po1 = closest(self,o.p); // 상대 중심과 내 도형의 위치(충돌-o.p,비충돌-내 도형의 경계) 저장 
        Point po2 = o.closest(self);
        if(o.collides(po1) || collides(self,po2)) // 내 도형의 중심이나 상대 도형의 경계가 내 도형과 충돌한다면 츙돌
            return true;
        else
            return false; 
    }    
        
    @Override
    String toSVG(Point self) 
    { 
        return "<polygon points='"+self.x+","+self.y+" "+ (self.x+w)+","+self.y +" "+self.x+","+(self.y +h)+"' style='stroke:black; stroke-width:1; fill:"+color+"; opacity:0.33' />";
    }

}

com.twosigma.beaker.javash.bkr31bc15ee.RTriangle

In [17]:
import com.twosigma.beakerx.widget.*;

RGB red = new RGB(255,0,0);
RGB blue = new RGB(0,0,255);
RGB magenta = new RGB(255,0,255);

RTriangle s1 = new RTriangle(100,100, red);
RTriangle s2 = new RTriangle(70,-70, blue);

Obj c1 = new Obj(new Point( 50, 50), s1);
Obj c2 = new Obj(new Point(110,110), s2);
Obj c3 = new Obj(new Point(160,160), s1);
Obj c4 = new Obj(new Point(220,220), s2);

System.out.println( c1 );
System.out.println( c2 );
System.out.println( c3 );
System.out.println( c4 );

System.out.println( c1.collides(c2) ); // 이런 결과가
System.out.println( c2.collides(c3) ); // 이런 결과가
System.out.println( c3.collides(c4) ); // 이런 결과가

System.out.println( c1.collides(c3) ); // 요런 결과가
System.out.println( c2.collides(c4) ); // 요런 결과가

HTML widget = new HTML();
String sgv1 =
    "<svg width='300' height='300' style='border:solid 1px'>"
    + c1 + c2 + c3 + c4 +
    "</svg>";

// s1 = new Circle(60, magenta);

widget.setValue("<div>"+sgv1+"</div>");
return widget; // 그림에 나타나는 것과 맞는지 시각적으로 확인해 볼 수 있도록 toSVG를 정의한 것이다

<polygon points='50,50 150,50 50,150' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' />
<polygon points='110,110 180,110 110,40' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' />
<polygon points='160,160 260,160 160,260' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' />
<polygon points='220,220 290,220 220,150' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' />
true
false
true
false
false


In [35]:
import java.util.Random;
import com.twosigma.beakerx.widget.*;

Random r = new Random();
RGB red = new RGB(255,0,0);
RGB blue = new RGB(0,0,255);

String sgvs[] = new String[100];
String vaild[] = new String[100];
for(int i=0; i<100;i++)
{
    RTriangle t1 = new RTriangle( r.nextInt(200)-100, r.nextInt(200)-100,red );
    RTriangle t2 = new RTriangle( r.nextInt(200)-100, r.nextInt(200)-100,blue );
    
    Obj o1 = new Obj(new Point( r.nextInt(200)+50, r.nextInt(200)+50 ), t1);
    Obj o2 = new Obj(new Point( r.nextInt(200)+50, r.nextInt(200)+50 ), t2);
    
    vaild[i] = i +" :" + o1.collides(o2);

    sgvs[i] =
    "<svg width='300' height='300' style='border:solid 1px'>"
    + o1 + o2 +
    "</svg>";
}
HTML widget = new HTML();
String wi = "<div>";
for(int i=0; i<100;i++)
{
    wi += sgvs[i];
    System.out.println(vaild[i]);
    
}
System.out.println(wi);
widget.setValue(wi+"</div>");
return widget;

0 :false
1 :false
2 :false
3 :false
4 :false
5 :false
6 :false
7 :false
8 :true
9 :true
10 :false
11 :false
12 :false
13 :false
14 :false
15 :false
16 :false
17 :false
18 :true
19 :false
20 :false
21 :false
22 :false
23 :false
24 :false
25 :false
26 :true
27 :false
28 :false
29 :false
30 :false
31 :false
32 :false
33 :false
34 :false
35 :false
36 :true
37 :false
38 :false
39 :false
40 :true
41 :false
42 :false
43 :false
44 :false
45 :false
46 :true
47 :false
48 :false
49 :false
50 :false
51 :false
52 :false
53 :false
54 :false
55 :false
56 :false
57 :false
58 :true
59 :false
60 :false
61 :false
62 :false
63 :false
64 :false
65 :false
66 :false
67 :false
68 :true
69 :false
70 :false
71 :false
72 :false
73 :false
74 :true
75 :false
76 :false
77 :false
78 :false
79 :false
80 :false
81 :false
82 :false
83 :false
84 :false
85 :false
86 :false
87 :true
88 :false
89 :false
90 :false
91 :false
92 :false
93 :false
94 :false
95 :false
96 :false
97 :false
98 :false
99 :false


<div><svg width='300' height='300' style='border:solid 1px'><polygon points='149,178 237,178 149,152' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='52,176 92,176 52,250' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='140,133 61,133 140,162' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='227,244 291,244 227,178' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='187,192 100,192 187,200' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='97,112 21,112 97,143' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='213,73 308,73 213,28' style='stroke:black; stroke-width:1; fill

style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='90,150 108,150 90,184' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='130,93 35,93 130,8' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='152,80 77,80 152,129' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='55,225 142,225 55,142' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='102,230 127,230 102,182' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='87,158 101,158 87,257' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='229,110 206,110 229,45' style='stroke:black; stroke-width:

In [39]:
import com.twosigma.beakerx.widget.*;
String wi = "<div><svg width='300' height='300' style='border:solid 1px'><polygon points='149,178 237,178 149,152' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='52,176 92,176 52,250' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='140,133 61,133 140,162' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='227,244 291,244 227,178' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='187,192 100,192 187,200' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='97,112 21,112 97,143' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='213,73 308,73 213,28' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='163,199 119,199 163,210' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='152,241 84,241 152,264' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='63,122 -7,122 63,119' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='141,122 112,122 141,119' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='175,216 81,216 175,255' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='215,191 136,191 215,109' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='189,91 191,91 189,44' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='120,190 207,190 120,223' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='207,153 144,153 207,175' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='216,249 237,249 216,165' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='202,233 267,233 202,327' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='120,152 63,152 120,57' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='183,153 109,153 183,103' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='88,96 97,96 88,139' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='242,150 232,150 242,55' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='247,191 251,191 247,149' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='56,92 -8,92 56,38' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='97,124 185,124 97,101' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='94,121 77,121 94,24' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='56,91 26,91 56,146' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='151,62 61,62 151,45' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='69,167 146,167 69,128' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='228,53 312,53 228,65' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='133,198 111,198 133,243' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='114,83 185,83 114,77' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='78,72 152,72 78,82' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='146,183 220,183 146,260' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='81,136 17,136 81,169' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='226,113 322,113 226,15' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='55,193 40,193 55,134' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='70,131 6,131 70,218' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='198,217 267,217 198,164' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='195,227 284,227 195,236' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='187,72 248,72 187,146' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='152,175 197,175 152,252' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='71,82 123,82 71,-2' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='109,190 44,190 109,110' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='188,224 181,224 188,307' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='117,189 211,189 117,185' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='87,131 128,131 87,191' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='141,247 63,247 141,300' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='229,89 195,89 229,16' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='116,92 160,92 116,189' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='191,93 163,93 191,1' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='211,162 178,162 211,185' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='144,122 168,122 144,32' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='124,86 207,86 124,6' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='235,80 164,80 235,172' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='117,148 51,148 117,93' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='111,150 144,150 111,129' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='235,122 278,122 235,40' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='93,61 40,61 93,145' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='177,161 206,161 177,153' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='118,123 114,123 118,111' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='56,139 -22,139 56,139' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='183,245 210,245 183,235' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='174,230 112,230 174,231' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='70,147 112,147 70,149' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='137,238 111,238 137,303' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='60,71 120,71 60,-7' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='91,142 134,142 91,198' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='130,125 193,125 130,84' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='133,96 44,96 133,94' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='142,68 199,68 142,43' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='117,199 101,199 117,176' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='237,84 210,84 237,102' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='234,84 172,84 234,48' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='176,102 197,102 176,182' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='228,54 147,54 228,-1' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='185,166 175,166 185,114' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='71,144 117,144 71,176' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='236,164 285,164 236,247' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='166,238 108,238 166,155' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='120,118 190,118 120,147' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='153,127 58,127 153,92' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='181,246 101,246 181,257' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='209,76 268,76 209,140' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='142,218 131,218 142,129' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='89,127 116,127 89,47' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='94,222 107,222 94,268' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='196,237 243,237 196,229' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='204,83 228,83 204,11' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='235,180 268,180 235,202' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='184,52 130,52 184,130' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='70,69 123,69 70,159' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='197,59 182,59 197,150' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='175,50 209,50 175,70' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='239,176 273,176 239,236' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='100,160 171,160 100,120' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='132,66 181,66 132,-15' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='76,166 25,166 76,195' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='133,67 61,67 133,138' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='126,156 134,156 126,213' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='83,199 159,199 83,212' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='231,197 207,197 231,147' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='100,125 88,125 100,157' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='210,148 300,148 210,100' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='109,108 144,108 109,138' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='55,162 80,162 55,127' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='239,103 307,103 239,95' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='246,206 159,206 246,201' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='108,185 63,185 108,274' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='91,123 34,123 91,55' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='240,73 175,73 240,128' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='241,141 301,141 241,73' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='122,119 102,119 122,47' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='97,99 26,99 97,44' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='50,212 69,212 50,130' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='90,150 108,150 90,184' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='130,93 35,93 130,8' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='152,80 77,80 152,129' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='55,225 142,225 55,142' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='102,230 127,230 102,182' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='87,158 101,158 87,257' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='229,110 206,110 229,45' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='139,239 227,239 139,220' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='237,241 158,241 237,317' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='83,96 -9,96 83,23' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='217,57 150,57 217,-23' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='126,87 86,87 126,173' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='229,164 139,164 229,165' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='65,221 111,221 65,310' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='57,166 47,166 57,80' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='181,203 230,203 181,155' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='236,162 204,162 236,189' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='99,245 130,245 99,197' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='51,194 -34,194 51,257' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='208,137 302,137 208,126' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='177,186 215,186 177,99' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='115,216 59,216 115,278' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='63,240 130,240 63,253' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='196,180 261,180 196,105' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='243,78 224,78 243,-21' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='61,64 26,64 61,20' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='138,174 221,174 138,79' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='66,147 63,147 66,107' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='191,60 156,60 191,89' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='98,180 106,180 98,92' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='179,160 190,160 179,94' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='197,247 177,247 197,173' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='85,217 72,217 85,142' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='205,174 129,174 205,75' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='217,138 179,138 217,68' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='191,160 167,160 191,137' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='195,72 202,72 195,81' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='217,76 275,76 217,29' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='194,113 214,113 194,17' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='227,94 296,94 227,190' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='111,54 181,54 111,-28' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='88,145 38,145 88,240' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='117,65 119,65 117,54' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='172,91 105,91 172,63' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='128,111 160,111 128,144' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='84,93 102,93 84,140' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='216,55 250,55 216,30' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='171,167 180,167 171,102' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='165,137 104,137 165,127' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='105,143 113,143 105,87' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='210,144 291,144 210,136' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='203,83 193,83 203,170' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='83,127 -17,127 83,160' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='96,242 172,242 96,317' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='81,135 64,135 81,110' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='208,126 165,126 208,188' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='194,111 190,111 194,95' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='189,164 180,164 189,70' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='123,98 85,98 123,88' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='179,153 123,153 179,245' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='136,228 232,228 136,218' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='201,166 180,166 201,162' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='160,141 120,141 160,128' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='118,77 153,77 118,39' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='235,60 234,60 235,19' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='239,50 180,50 239,92' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='213,100 134,100 213,127' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='229,131 324,131 229,188' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='154,146 177,146 154,60' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='113,232 22,232 113,269' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='129,236 219,236 129,301' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='50,82 -46,82 50,65' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='244,227 203,227 244,236' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='243,191 242,191 243,163' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='156,97 222,97 156,4' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='168,188 216,188 168,268' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='174,187 83,187 174,93' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='78,217 66,217 78,234' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='162,80 218,80 162,148' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='88,129 0,129 88,34' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='232,192 151,192 232,273' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='86,70 -2,70 86,146' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='91,222 45,222 91,256' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg><svg width='300' height='300' style='border:solid 1px'><polygon points='93,216 82,216 93,189' style='stroke:black; stroke-width:1; fill:rgb(255,0,0); opacity:0.33' /><polygon points='240,222 334,222 240,161' style='stroke:black; stroke-width:1; fill:rgb(0,0,255); opacity:0.33' /></svg></div>";

HTML widget = new HTML();
widget.setValue(wi);
return widget;
//34, 59 fail
// 랜덤 100 : 충돌 13 : 검출 11


0 :false
1 :false
2 :false
3 :false
4 :false
5 :false
6 :false
7 :false
8 :true
9 :true
10 :false
11 :false
12 :false
13 :false
14 :false
15 :false
16 :false
17 :false
18 :true
19 :false
20 :false
21 :false
22 :false
23 :false
24 :false
25 :false
26 :true
27 :false
28 :false
29 :false
30 :false
31 :false
32 :false
33 :false
34 :false
35 :false
36 :true
37 :false
38 :false
39 :false
40 :true
41 :false
42 :false
43 :false
44 :false
45 :false
46 :true
47 :false
48 :false
49 :false
50 :false
51 :false
52 :false
53 :false
54 :false
55 :false
56 :false
57 :false
58 :true
59 :false
60 :false
61 :false
62 :false
63 :false
64 :false
65 :false
66 :false
67 :false
68 :true
69 :false
70 :false
71 :false
72 :false
73 :false
74 :true
75 :false
76 :false
77 :false
78 :false
79 :false
80 :false
81 :false
82 :false
83 :false
84 :false
85 :false
86 :false
87 :true
88 :false
89 :false
90 :false
91 :false
92 :false
93 :false
94 :false
95 :false
96 :false
97 :false
98 :false
99 :false